In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import sys
import math
from netCDF4 import Dataset
import os, calendar, sys, fnmatch, datetime

# Load Huang et al. 2023 Ellipsoidal dust optical properties files

In [2]:
# ===== Aerosol (e.g., dust) optical properties (i.e., Qext, SSA, g) =====
# ***Note: The order of 14 bands is listed in TABLE 1 in RRTM_SW instruction (It is not monotonically increase or decrease)
# ***Note: The order of spectral optical properties should be consistent with the order in TABLE 1

def process_optical_properties(wavelength_band, txt_files):
    wavelength = wavelength_range[wavelength_band]
    print(txt_files[wavelength_band])
    opt_prop_sw_original = np.loadtxt(opt_dir + '{}'.format(txt_files[wavelength_band]))
    print(opt_prop_sw_original.shape)

    wl = np.array(opt_prop_sw_original[:, 0][:])
    dust_d = np.array(opt_prop_sw_original[:, 1][:])
    #size_parameter = np.pi * dust_d / wl #f'size_parameter_{wavelength:.4f}': size_parameter,
    n_RI = np.array(opt_prop_sw_original[:, 2][:])
    k_RI = np.array(opt_prop_sw_original[:, 3][:])
    
    qe_dust_sw = opt_prop_sw_original[:, 4]
    ssa_dust_sw = opt_prop_sw_original[:, 5]
    g_dust_sw = opt_prop_sw_original[:, 6]

    # Create variable names dynamically
    variables = {
        f'wl_{wavelength:.4f}': wl,
        f'dust_d_{wavelength:.4f}': dust_d,
        f'n_RI_{wavelength:.4f}': n_RI,
        f'k_RI_{wavelength:.4f}': k_RI,
        f'qe_dust_sw_{wavelength:.4f}': qe_dust_sw,
        f'ssa_dust_sw_{wavelength:.4f}': ssa_dust_sw,
        f'g_dust_sw_{wavelength:.4f}': g_dust_sw,
    }
    return np.array((wl,dust_d,n_RI,k_RI,qe_dust_sw,ssa_dust_sw,g_dust_sw)) #variables

# Wavelength range and files
opt_dir = 'SW_Huang_Dust_Optics/'
wavelength_range = [0.35, 0.55, 0.75, 0.95, 2, 4, 6, 8, 10, 12, 14, 16, 35]
Huang_2023_dust_opt_prop_txt_files = ['shapein_TAMU_0.3500.txt', 'shapein_TAMU_0.5500.txt', 'shapein_TAMU_0.7500.txt', 'shapein_TAMU_0.9500.txt', 'shapein_TAMU_2.0000.txt', 'shapein_TAMU_4.0000.txt', 'shapein_TAMU_6.0000.txt', 'shapein_TAMU_8.0000.txt', 'shapein_TAMU_10.0000.txt', 'shapein_TAMU_12.0000.txt', 'shapein_TAMU_14.0000.txt', 'shapein_TAMU_16.0000.txt', 'shapein_TAMU_35.0000.txt']

# Example usage for wavelength_band 0
#wavelength_band = 0 #0.35 micrometers
#wavelength_band = 1 #0.55 micrometers
#wavelength_band = 2 #0.75 micrometers
#wavelength_band = 3 #0.95 micrometers
#wavelength_band = 4 #2 micrometers
#wavelength_band = 5 #4 micrometers
#wavelength_band = 6 #6 micrometers
#wavelength_band = 7 #8 micrometers
#wavelength_band = 8 #10 micrometers
#wavelength_band = 9 #12 micrometers
#wavelength_band = 10 #14 micrometers
#wavelength_band = 11 #16 micrometers
#wavelength_band = 12 #35 micrometers

#wavelength = wavelength_range[wavelength_band]

In [3]:
# Wavelength range and files
opt_dir = 'SW_Huang_Dust_Optics/'
wavelength_range = [0.35, 0.55, 0.75, 0.95, 2, 4, 6, 8, 10, 12, 14, 16, 35]
Huang_2023_dust_opt_prop_txt_files = ['shapein_TAMU_0.3500.txt', 'shapein_TAMU_0.5500.txt', 'shapein_TAMU_0.7500.txt', 'shapein_TAMU_0.9500.txt', 'shapein_TAMU_2.0000.txt', 'shapein_TAMU_4.0000.txt', 'shapein_TAMU_6.0000.txt', 'shapein_TAMU_8.0000.txt', 'shapein_TAMU_10.0000.txt', 'shapein_TAMU_12.0000.txt', 'shapein_TAMU_14.0000.txt', 'shapein_TAMU_16.0000.txt', 'shapein_TAMU_35.0000.txt']

# Array to store results
opt_prop_sw_array = np.empty((len(wavelength_range),), dtype=object)

# Process optical properties for each wavelength band
for i in range(len(wavelength_range)):
    opt_prop_sw = process_optical_properties(i, Huang_2023_dust_opt_prop_txt_files)
    opt_prop_sw_array[i] = opt_prop_sw

# Accessing the results
#for i, opt_prop_sw in enumerate(opt_prop_sw_array):
#    print(f"Example usage for wavelength_band {wavelength_range[i]}: {opt_prop_sw}")

shapein_TAMU_0.3500.txt
(14400, 9)
shapein_TAMU_0.5500.txt
(14400, 9)
shapein_TAMU_0.7500.txt
(14400, 9)
shapein_TAMU_0.9500.txt
(14400, 9)
shapein_TAMU_2.0000.txt
(18000, 9)
shapein_TAMU_4.0000.txt
(18000, 9)
shapein_TAMU_6.0000.txt
(18000, 9)
shapein_TAMU_8.0000.txt
(18000, 9)
shapein_TAMU_10.0000.txt
(18000, 9)
shapein_TAMU_12.0000.txt
(18000, 9)
shapein_TAMU_14.0000.txt
(18000, 9)
shapein_TAMU_16.0000.txt
(18000, 9)
shapein_TAMU_35.0000.txt
(18000, 9)


# Calculate DustCOMM mean geometric diamater of each bin

In [4]:
#----------
# using DustCOMM 2021 papers b) https://dustcomm.atmos.ucla.edu/
# specifically https://dustcomm.atmos.ucla.edu/data/K21b/ dataset DustCOMM_source_region_DAOD_seas_bin_abs.nc
# assumes aspherical dust shape, PM20 dust
#----------

daod_Dustcomm_PM20_Dataset = Dataset('DustCOMM_source_region_DAOD_annual_bin_abs.nc')
print(daod_Dustcomm_PM20_Dataset) #[season,source,diameter,lat,lon,bin]

lat_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lat'][:])
lon_Dustcomm_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['lon'][:])
mean_Daod_Dustcomm_20PM = np.array(daod_Dustcomm_PM20_Dataset.variables['Mean'][:])
#dustcomm_seas = np.array(daod_Dustcomm_PM20_Dataset.variables['season'][:])
dustcomm_Sources_PM20 = np.array(daod_Dustcomm_PM20_Dataset.variables['source'][:]) 
dustcomm_bin_lower = np.array(daod_Dustcomm_PM20_Dataset.variables['bin_D_lower'][:])
dustcomm_bin_upper = np.array(daod_Dustcomm_PM20_Dataset.variables['bin_D_upper'][:])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): source(9), diameter(6), lat(96), lon(144), bin(6)
    variables(dimensions): float64 source(source), float64 bin_D_lower(diameter), float64 bin_D_upper(diameter), float64 lat(lat), float64 lon(lon), float64 Median(lat, lon, bin, source), float64 Mean(lat, lon, bin, source), float64 Neg1sigma(lat, lon, bin, source), float64 Pos1sigman(lat, lon, bin, source), float64 Neg2sigma(lat, lon, bin, source), float64 Pos2sigma(lat, lon, bin, source)
    groups: 


In [5]:
#----------
# using DustCOMM 2021 papers b) https://dustcomm.atmos.ucla.edu/
# specifically https://dustcomm.atmos.ucla.edu/data/K21b/ dataset DustCOMM_source_region_DAOD_seas_bin_abs.nc
# assumes aspherical dust shape, PM20 dust
#----------

loading_Dustcomm_PM20_Dataset = Dataset('DustCOMM_source_region_dust_loading_annual_bin_abs.nc')
print(loading_Dustcomm_PM20_Dataset) #[season,source,diameter,lat,lon,bin]

loading_lat_Dustcomm_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['lat'][:])
loading_lon_Dustcomm_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['lon'][:])
mean_loading_Dustcomm_20PM = np.array(loading_Dustcomm_PM20_Dataset.variables['Mean'][:])
#dustcomm_seas = np.array(daod_Dustcomm_PM20_Dataset.variables['season'][:])
loading_dustcomm_Sources_PM20 = np.array(loading_Dustcomm_PM20_Dataset.variables['source'][:]) 
loading_dustcomm_bin_lower = np.array(loading_Dustcomm_PM20_Dataset.variables['bin_D_lower'][:])
loading_dustcomm_bin_upper = np.array(loading_Dustcomm_PM20_Dataset.variables['bin_D_upper'][:])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    dimensions(sizes): source(9), diameter(6), lat(96), lon(144), bin(6)
    variables(dimensions): float64 source(source), float64 bin_D_lower(diameter), float64 bin_D_upper(diameter), float64 lat(lat), float64 lon(lon), float64 Median(lat, lon, bin, source), float64 Mean(lat, lon, bin, source), float64 Neg1sigma(lat, lon, bin, source), float64 Pos1sigman(lat, lon, bin, source), float64 Neg2sigma(lat, lon, bin, source), float64 Pos2sigma(lat, lon, bin, source)
    groups: 


In [6]:
print(dustcomm_bin_lower)
print(dustcomm_bin_upper)

[ 0.2  0.5  1.   2.5  5.  10. ]
[ 0.5  1.   2.5  5.  10.  20. ]


In [7]:
mean_geometric_diameter_bin = np.sqrt(dustcomm_bin_lower*dustcomm_bin_upper)

In [8]:
print(mean_geometric_diameter_bin)

[ 0.31622777  0.70710678  1.58113883  3.53553391  7.07106781 14.14213562]


# Choose Huang et al. 2023 dust optical properties that best match Adebiyi RI's, and DustCOMM mean geomtric diameter

In [9]:
#grid_box = [0,18] #0W, 18N
North_African_dust_adebiyi_k = 0.0012
North_African_dust_adebiyi_nr = 1.51

use DustCOMM Saharan and Sahelian dust partition in a grid box, in the DAOD. sum the two sahara sources, to calculate my own spatially resolved RI (Later)

match_d_opt_prop_sw = []
match_n_RI_opt_prop_sw = []
match_k_RI_opt_prop_sw = []
n_RI_index = []
k_RI_index = []North African dust has a mean imaginary refractive index at 550 nm wavelength of 0.0012 (one standard error range of 0.0009–0.0016
and a real dust refractive index, nr = 1.51 ± 0.03 (which is the same for Sahara and Sahel) (Di Biagio et al. 2019)
We assume that RI is constant across particle size (Adebiyi et al. 2023, Obiso et al. 2023)

In [10]:
#match_d_opt_prop_sw = []
#match_n_RI_opt_prop_sw = []
#match_k_RI_opt_prop_sw = []
#n_RI_index = []
#k_RI_index = []

print(opt_prop_sw_array[0][1]) #.shape # dust diameter
dust_d_list = list(opt_prop_sw_array[0][1])
#print(dust_d_list)

[ 0.1     0.1023  0.1047 ... 67.5521 68.7652 70.    ]


In [11]:
mean_geometric_diameter_bin_match = np.zeros(len(wavelength_range), dtype=object)
wl_mean_geometric_diameter_bin_match = []
index_dust_d = np.zeros(len(wavelength_range), dtype=object)
dust_d_match = [] #np.zeros(len(wavelength_range))

for iwl in range(0,len(wavelength_range)):
    for i in opt_prop_sw_array[iwl][1]:
        for j in mean_geometric_diameter_bin:
            if np.round(j, decimals=1) == np.round(i, decimals=1):
                mean_geometric_diameter_bin_match[iwl] = np.append(mean_geometric_diameter_bin_match[iwl],j)
                index_dust_d[iwl] = np.append(index_dust_d[iwl],dust_d_list.index(i))
                dust_d_match.append(i)



In [12]:
#print(dust_d_match)
print(len(dust_d_match))
print(mean_geometric_diameter_bin_match.shape)

20496
(13,)


In [13]:
def extract_unique_values(input_array):
    unique_values = np.unique(input_array)
    return unique_values

In [14]:
#print(index_dust_d)
unique_dust_d_index = np.zeros(len(opt_prop_sw_array), dtype=object)
for iwl in range(0,len(opt_prop_sw_array)):
    unique_dust_d_index[iwl] = extract_unique_values(index_dust_d[iwl]) #[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 81, 82, 83, 84, 85, 86, 87, 119, 120, 121, 154, 184, 214]
print(len(unique_dust_d_index),unique_dust_d_index)

13 [array([  0,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  81,  82,  83,  84,  85,  86,  87, 119, 120, 121,
        154, 184, 214])
 array([  0,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  81,  82,  83,  84,  85,  86,  87, 119, 120, 121,
        154, 184, 214])
 array([  0,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  81,  82,  83,  84,  85,  86,  87, 119, 120, 121,
        154, 184, 214])
 array([  0,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  81,  82,  83,  84,  85,  86,  87, 119, 120, 121,
        154, 184, 214])
 array([  0,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  81,  82,  83,  84,  85,  86,  87, 119, 120, 121,
        154, 184, 214])
 array([  0,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  81,  82,  83,  84,  85,  86,  87, 119, 120, 121,
 

In [15]:
#Make list of Yue dust diameters that match with DustCOMM mean geometric bin diameter
#Choosing 0 first wavelength 0.35 µm since all the indexes across wavelengths match --> unique_dust_d_index[0]
list_d_match = []
for i in range(0,len(unique_dust_d_index[0])):
    list_d_match.append(opt_prop_sw[1][unique_dust_d_index[0][i]])
#list_d_match = [0.2523, 0.2582, 0.2643, 0.2704, 0.2768, 0.2832, 0.2899, 0.2967, 0.3036, 0.3107, 0.318, 0.3254, 0.333, 0.3408, 0.3488, 0.6515, 0.6667, 0.6823, 0.6983, 0.7147, 0.7314, 0.7485, 1.5694, 1.6061, 1.6437, 3.5271, 7.0609, 14.1354]
print(len(list_d_match), list_d_match)

29 [0.1, 0.2523, 0.2582, 0.2643, 0.2704, 0.2768, 0.2832, 0.2899, 0.2967, 0.3036, 0.3107, 0.318, 0.3254, 0.333, 0.3408, 0.3488, 0.6515, 0.6667, 0.6823, 0.6983, 0.7147, 0.7314, 0.7485, 1.5694, 1.6061, 1.6437, 3.5271, 7.0609, 14.1354]


# Need to change to not be brute force

## To find best matching d_dust[i] to geometric diameter for each particle bin

In [16]:
def find_closest_value(target, array):
    # Check if the array is empty
    if not array:
        return None

    # Initialize variables to store the closest value and its difference
    closest_value = array[0]
    min_difference = abs(target - array[0])

    # Iterate through the array
    for element in array:
        # Calculate the absolute difference between the target and the current element
        difference = abs(target - element)

        # Update the closest value if the current element is closer
        if difference < min_difference:
            min_difference = difference
            closest_value = element

    return closest_value

# Example usage
target_number = 7
my_array = [1, 3, 5, 8, 10]

result = find_closest_value(target_number, my_array)
print(f"The value in the array closest to {target_number} is: {result}")

The value in the array closest to 7 is: 8


In [17]:
#choosing the best matching diameters to the DustCOMM mean geometric diameters of each bin
new_geometric_mean_diameter_matching = np.zeros(len(mean_geometric_diameter_bin), dtype=object)
index_new_geometric_mean_diameter_matching = np.zeros(len(mean_geometric_diameter_bin), dtype=object)
list_d_match = list(list_d_match)

for j in range(0,len(mean_geometric_diameter_bin)):
    new_geometric_mean_diameter_matching[j] = find_closest_value(mean_geometric_diameter_bin[j], list_d_match)
    index_new_geometric_mean_diameter_matching[j] = unique_dust_d_index[0][list_d_match.index(new_geometric_mean_diameter_matching[j])]
print(new_geometric_mean_diameter_matching)
print(index_new_geometric_mean_diameter_matching)

[0.318 0.7147 1.5694 3.5271 7.0609 14.1354]
[50 85 119 154 184 214]


In [18]:
print(opt_prop_sw_array[0].shape)
print(opt_prop_sw_array.shape)
print(len(opt_prop_sw_array[0]))
print(len(opt_prop_sw_array[1][1][:]))

(7, 14400)
(13,)
7
14400


In [19]:
avg_geo_d_opt_prop_sw = np.zeros(len(opt_prop_sw_array), dtype=object)

for iwl in range(len(opt_prop_sw_array)):
    for i in new_geometric_mean_diameter_matching:
        temp_array = []
        for j in range(len(opt_prop_sw_array[0][0])):
            if opt_prop_sw_array[iwl][1, j] == i:
                temp_array.extend([opt_prop_sw_array[iwl][:, j]])
        avg_geo_d_opt_prop_sw[iwl] = np.array(temp_array)

print(avg_geo_d_opt_prop_sw.shape)
print(avg_geo_d_opt_prop_sw[0].shape)
print(avg_geo_d_opt_prop_sw[0])

(13,)
(48, 7)
[[3.500000e-01 1.413540e+01 1.450000e+00 5.000000e-04 2.888290e+00
  9.020188e-01 8.568293e-01]
 [3.500000e-01 1.413540e+01 1.450000e+00 1.000000e-03 2.884958e+00
  8.337529e-01 8.815522e-01]
 [3.500000e-01 1.413540e+01 1.450000e+00 2.000000e-03 2.871925e+00
  7.460455e-01 9.055269e-01]
 [3.500000e-01 1.413540e+01 1.450000e+00 3.000000e-03 2.865846e+00
  6.951403e-01 9.223020e-01]
 [3.500000e-01 1.413540e+01 1.450000e+00 4.000000e-03 2.862037e+00
  6.591344e-01 9.357687e-01]
 [3.500000e-01 1.413540e+01 1.450000e+00 5.000000e-03 2.887351e+00
  6.308029e-01 9.472149e-01]
 [3.500000e-01 1.413540e+01 1.450000e+00 6.000000e-03 2.878176e+00
  6.170383e-01 9.499616e-01]
 [3.500000e-01 1.413540e+01 1.450000e+00 1.000000e-02 2.886860e+00
  5.782538e-01 9.582366e-01]
 [3.500000e-01 1.413540e+01 1.500000e+00 5.000000e-04 2.911882e+00
  8.991086e-01 8.430883e-01]
 [3.500000e-01 1.413540e+01 1.500000e+00 1.000000e-03 2.912696e+00
  8.299420e-01 8.700321e-01]
 [3.500000e-01 1.413540e+0

In [20]:

n_RI_geo = np.zeros(len(opt_prop_sw_array), dtype=object)
k_RI_geo = np.zeros(len(opt_prop_sw_array), dtype=object)
qe_dust_sw_geo = np.zeros(len(opt_prop_sw_array), dtype=object)
ssa_dust_sw_geo = np.zeros(len(opt_prop_sw_array), dtype=object)
g_dust_sw_geo = np.zeros(len(opt_prop_sw_array), dtype=object)

for iwl in range(0,len(opt_prop_sw_array)):
    n_RI_geo[iwl] = avg_geo_d_opt_prop_sw[iwl][:2]
    k_RI_geo[iwl] = avg_geo_d_opt_prop_sw[iwl][:,3]
    qe_dust_sw_geo[iwl] = avg_geo_d_opt_prop_sw[iwl][:,4]
    ssa_dust_sw_geo[iwl] = avg_geo_d_opt_prop_sw[iwl][:,5]
    g_dust_sw_geo[iwl] = avg_geo_d_opt_prop_sw[iwl][:,6]
#print(k_RI_geo.shape,k_RI_geo)

## 1/16 Rewrite cell below to write new array for each 13 wavelengths with matching RI's (n and k)

In [21]:
n_RI_geo_list = list(n_RI_geo)
#print(n_RI_geo_list)
match_n_RI_sw = np.zeros(len(opt_prop_sw_array), dtype=object) #[]
index_match_n_RI_sw = np.zeros(len(opt_prop_sw_array), dtype=object) #[]
match_n_RI_opt_prop_sw = np.zeros(len(opt_prop_sw_array), dtype=object) #[]

for iwl in range(0,len(opt_prop_sw_array)):
    for n in n_RI_geo[iwl]:
        for m in k_RI_match_n[iwl]:
            if np.round(m, decimals=3) == np.round(North_African_dust_adebiyi_k, decimals=3):
                if np.round(n, decimals=1) == np.round(North_African_dust_adebiyi_nr, decimals=1):
                    match_n_RI_sw[iwl] = np.append(match_n_RI_sw[iwl],n)
                    index_match_n_RI_sw[iwl] = np.append(match_n_RI_sw[iwl],n_RI_geo_list.index(n))
print(len(match_n_RI_sw),match_n_RI_sw)
match_n_RI_sw = np.array(match_n_RI_sw)
index_match_n_RI_sw = np.array(index_match_n_RI_sw)
print(index_match_n_RI_sw)

unique_index_match_n_RI_sw = extract_unique_values(index_match_n_RI_sw[0])
print('unique_index_match_n_RI_sw: ',unique_index_match_n_RI_sw)

#turn into appending for loop
new_n_RI_geo = np.zeros(len(opt_prop_sw_array), dtype=object) #[]

for iwl in range(0,len(opt_prop_sw_array)):
    for i in range(0,len(unique_index_match_n_RI_sw)):
        new_n_RI_geo[iwl] = np.append(n_RI_geo[unique_index_match_n_RI_sw[i]])

for i in new_n_RI_geo:
    for j in range(0,len(avg_geo_d_opt_prop_sw)):
        if avg_geo_d_opt_prop_sw[:,2][j] == i:
            match_n_RI_opt_prop_sw.append((avg_geo_d_opt_prop_sw[j,:]))
            
match_n_RI_opt_prop_sw = np.array (match_n_RI_opt_prop_sw)
#print(match_n_RI_opt_prop_sw.shape,match_n_RI_opt_prop_sw[0])

NameError: name 'k_RI_match_n' is not defined

In [ ]:
n_RI_match_n = match_n_RI_opt_prop_sw[:,2]
k_RI_match_n = match_n_RI_opt_prop_sw[:,3]
qe_dust_sw_match_n = match_n_RI_opt_prop_sw[:,4]
ssa_dust_sw_match_n = match_n_RI_opt_prop_sw[:,5]
g_dust_sw_match_n = match_n_RI_opt_prop_sw[:,6]
#print(k_RI_match_n.shape,k_RI_geo)

In [ ]:
k_RI_n_match_list = list(k_RI_match_n)
#print(k_RI_geo_list)
match_k_RI_sw = []
index_match_k_RI_sw = []
match_k_RI_opt_prop_sw = []

for m in k_RI_match_n:
    if np.round(m, decimals=3) == np.round(North_African_dust_adebiyi_k, decimals=3):
        match_k_RI_sw.append(m)
        index_match_k_RI_sw.append(k_RI_n_match_list.index(m))
print(len(match_k_RI_sw),match_k_RI_sw)
match_k_RI_sw = np.array(match_k_RI_sw)
index_match_k_RI_sw = np.array(index_match_k_RI_sw)
print(len(index_match_k_RI_sw), index_match_k_RI_sw)

unique_index_match_k_RI_sw = extract_unique_values(index_match_k_RI_sw)
#turn into appending for loop
new_k_RI_geo = []
for i in range(0,len(unique_index_match_n_RI_sw)):
    new_n_RI_geo.append(k_RI_geo[unique_index_match_n_RI_sw[i]])

#for i in new_k_RI_geo:
for j in range(0,len(match_n_RI_opt_prop_sw)):
    if match_n_RI_opt_prop_sw[j,3] == k_RI_geo[1]:
        match_k_RI_opt_prop_sw.append((match_n_RI_opt_prop_sw[j,:]))

match_k_RI_opt_prop_sw = np.array(match_k_RI_opt_prop_sw)
#print(match_k_RI_opt_prop_sw.shape, match_k_RI_opt_prop_sw)

In [ ]:
bins_opt_prop_sw = []

def process_bin(bin_number, matching_array):
    bin_opt_prop_sw = []
    for j in range(0, len(match_k_RI_opt_prop_sw)):
        if matching_array[:, 1][j] == new_geometric_mean_diameter_matching[bin_number]:
            bin_opt_prop_sw.append(matching_array[j, :])
    return bin_opt_prop_sw

# Process each bin and append the result to the list
for i in range(0,len(mean_geometric_diameter_bin)):
    bins_opt_prop_sw.append(process_bin(i, match_k_RI_opt_prop_sw))
bins_opt_prop_sw = np.array(bins_opt_prop_sw)

# Print the results
for i, bin_opt_prop_sw in enumerate(bins_opt_prop_sw, start=1):
    print(f"bin{i}_opt_prop_sw: {bin_opt_prop_sw}")

In [ ]:
print(bins_opt_prop_sw.shape)

In [ ]:
len(bins_opt_prop_sw)

## Interpolate diamters to mean geometric diameter, k from 0.001 to 0.0012, and n from 1.50 to 1.51

In [ ]:
def calculate_bin_average(array,bin_data):
    ibin = len(array)
    if ibin == 6:
        return bin_data[0]
    else:
        return (bin_data[0] + bin_data[1]) / 2

# Calculate and print averages for each bin
avg_bin1_opt_prop_sw = calculate_bin_average(bins_opt_prop_sw,bins_opt_prop_sw[0])
print(f"Average for bin1_opt_prop_sw: {avg_bin1_opt_prop_sw}")

avg_bin2_opt_prop_sw = calculate_bin_average(bins_opt_prop_sw,bins_opt_prop_sw[1])
print(f"Average for bin2_opt_prop_sw: {avg_bin2_opt_prop_sw}")

avg_bin3_opt_prop_sw = calculate_bin_average(bins_opt_prop_sw,bins_opt_prop_sw[2])
print(f"Average for bin3_opt_prop_sw: {avg_bin3_opt_prop_sw}")

avg_bin4_opt_prop_sw = calculate_bin_average(bins_opt_prop_sw,bins_opt_prop_sw[3])
print(f"Average for bin4_opt_prop_sw: {avg_bin4_opt_prop_sw}")

avg_bin5_opt_prop_sw = calculate_bin_average(bins_opt_prop_sw,bins_opt_prop_sw[4])
print(f"Average for bin5_opt_prop_sw: {avg_bin5_opt_prop_sw}")

avg_bin6_opt_prop_sw = calculate_bin_average(bins_opt_prop_sw,bins_opt_prop_sw[5])
print(f"Average for bin6_opt_prop_sw: {avg_bin6_opt_prop_sw}")


In [ ]:
# RRTM SW bands - average wavelength of each band (in RRTM order)
RRTM_SW_bands = [3.4615, 2.7885, 2.3255, 2.0465, 1.784, 1.4625, 1.2705, 1.01, 0.7015, 0.5335, 0.3935, 0.304, 0.2315, 8.0205]
RRTM_SW_bands_sorted = np.sort(RRTM_SW_bands)

Meeting Notes 1/4/24:

Look at how solar zenith angle variation effects "DRE" (difference between the two fluxes)
Send Jasper bulk dust optical properties by end of day tomorrow (plus text file)
write script to run SW RRTM for different solar zenith angles

In [ ]:
size_parameter_geometric_diameter_bin = np.pi*mean_geometric_diameter_bin/wavelength
print(size_parameter_geometric_diameter_bin)

## 1/9/24 I am having issues interpolating because interpolation sorts the variables from min to max

In [ ]:
def interpolate_bin_size_param(bin_data, bin_number):
    x = np.array([np.pi*bin_data[1]/bin_data[0], bin_data[2], bin_data[3]]) #(size parameter, real RI, imaginary RI)
    #print(x)
    y = np.array([bin_data[4], bin_data[5], bin_data[6]]) #(Q_e, SSA, g)
    print(y)

    x_interp = np.array([size_parameter_geometric_diameter_bin[bin_number-1],
                         North_African_dust_adebiyi_nr,
                         North_African_dust_adebiyi_k])

    x_sorted = np.sort(x)
    x_interp_sorted = np.sort(x_interp)

    # Interpolate the value of y at x_interp
    y_interp = np.interp(x_interp_sorted, x_sorted, y)

    # Restore the original order of y
    original_order_y = np.argsort(x_sorted)
    y_interp_original_order = y_interp[original_order_y]
    x_interp_original_order = x_interp[original_order_y]

    print(f"Interpolated Values of y{bin_number} at {x_interp_original_order}: {y_interp_original_order}")
    return x_interp_original_order, y_interp_original_order

# bin1
bin1_opt_prop_interp_size_param_x, bin1_opt_prop_interp_size_param_y = interpolate_bin_size_param(
    avg_bin1_opt_prop_sw, 1)

# bin2
bin2_opt_prop_interp_size_param_x, bin2_opt_prop_interp_size_param_y = interpolate_bin_size_param(
    avg_bin2_opt_prop_sw, 2)

# bin3
bin3_opt_prop_interp_size_param_x, bin3_opt_prop_interp_size_param_y = interpolate_bin_size_param(
    avg_bin3_opt_prop_sw, 3)

# bin4
bin4_opt_prop_interp_size_param_x, bin4_opt_prop_interp_size_param_y = interpolate_bin_size_param(
    avg_bin4_opt_prop_sw, 4)

# bin5
bin5_opt_prop_interp_size_param_x, bin5_opt_prop_interp_size_param_y = interpolate_bin_size_param(
    avg_bin5_opt_prop_sw, 5)

# bin6
bin6_opt_prop_interp_size_param_x, bin6_opt_prop_interp_size_param_y = interpolate_bin_size_param(
    avg_bin6_opt_prop_sw, 6)

# Calculate bulk dust Optical properties for (0W, 18N)

Reorder dust optical properties so it is in order (Q_e, ssa, g)

In [ ]:
# Calculate and print for each bin
all_bins_opt_prop_array = []

# Convert the list to a NumPy array
all_bins_opt_prop_array = np.array((bin1_opt_prop_interp_size_param_y, bin2_opt_prop_interp_size_param_y, bin3_opt_prop_interp_size_param_y, bin4_opt_prop_interp_size_param_y, bin5_opt_prop_interp_size_param_y, bin6_opt_prop_interp_size_param_y))
print(all_bins_opt_prop_array)

In [ ]:
print(loading_lat_Dustcomm_PM20.shape)
print(loading_lon_Dustcomm_PM20.shape)
print(mean_loading_Dustcomm_20PM.shape)
print(mean_loading_Dustcomm_20PM[0,0,1,2])
#dimensions(sizes): source(9), diameter(6), lat(96), lon(144), bin(6)
index_0W = []
index_18N = []

lat_0W = 0
lon_18N = 18

for ilat in range(0,len(loading_lat_Dustcomm_PM20)):
    if np.round(loading_lat_Dustcomm_PM20[ilat]) -1 == lat_0W:
        print(loading_lat_Dustcomm_PM20[ilat])
        index_0W.append(ilat)

for ilon in range(0,len(loading_lon_Dustcomm_PM20)):
    if np.round(loading_lon_Dustcomm_PM20[ilon]) == lon_18N: 
        print(loading_lon_Dustcomm_PM20[ilon])
        index_18N.append(ilon)
print(index_0W,index_18N)

In [ ]:
total_North_African_loading = mean_loading_Dustcomm_20PM[:,:,:,0] + mean_loading_Dustcomm_20PM[:,:,:,1] + mean_loading_Dustcomm_20PM[:,:,:,2]
Sarahan_laoding = mean_loading_Dustcomm_20PM[:,:,:,0] + mean_loading_Dustcomm_20PM[:,:,:,1]
Sahelian_loading = mean_loading_Dustcomm_20PM[:,:,:,2]
print(total_North_African_loading.shape)

In [ ]:
#ρ_d=(2.5±0.2)×10^3 kg m-3 is the globally representative density of dust aerosols (Fratini et al., 2007; Reid et al., 2008; Kaaden et al., 2009; Sow et al., 2009)
ρ_d = 2.5e3
delta_m_Db = total_North_African_loading[48,78,:]
print('delta_m_Db',delta_m_Db)

Q_e_bulk_0W_18N_bin_num = np.zeros(6)
Q_e_bulk_0W_18N_bin_den = np.zeros(6)
SSA_bulk_0W_18N_bin_num = np.zeros(6)
SSA_bulk_0W_18N_bin_den = np.zeros(6)
Q_scat = np.zeros(6)
g_bulk_0W_18N_bin_num = np.zeros(6)
g_bulk_0W_18N_bin_den = np.zeros(6)

#all_bins_opt_prop_array[i][0] = Q_e of bin i
#all_bins_opt_prop_array[i][1] = SSA of bin i
#all_bins_opt_prop_array[i][2] = g of bin i

for i in range(0,len(all_bins_opt_prop_array)):
    Q_e_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    Q_e_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2))/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    
    SSA_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0]*all_bins_opt_prop_array[i][1])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    SSA_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*all_bins_opt_prop_array[i][0])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    
    Q_scat[i] = all_bins_opt_prop_array[i][1] * all_bins_opt_prop_array[i][0]
    
    g_bulk_0W_18N_bin_num[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*Q_scat[i]*all_bins_opt_prop_array[i][2])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    g_bulk_0W_18N_bin_den[i] = (delta_m_Db[i]*(np.pi/4)*(mean_geometric_diameter_bin[i]**2)*Q_scat[i])/((np.pi/6)*(mean_geometric_diameter_bin[i]**3)*ρ_d)
    

In [ ]:
# Calculate values
Q_e_bulk_0W_18N_bulk = np.sum(Q_e_bulk_0W_18N_bin_num) / np.sum(Q_e_bulk_0W_18N_bin_den)
SSA_bulk_0W_18N_bulk = np.sum(SSA_bulk_0W_18N_bin_num) / np.sum(SSA_bulk_0W_18N_bin_den)
g_bulk_0W_18N_bulk = np.sum(g_bulk_0W_18N_bin_num) / np.sum(g_bulk_0W_18N_bin_den)

# Print the values
print("Q_e_bulk_0W_18N_bulk: ", Q_e_bulk_0W_18N_bulk)
print("SSA_bulk_0W_18N_bulk: ", SSA_bulk_0W_18N_bulk)
print("g_bulk_0W_18N_bulk: ", g_bulk_0W_18N_bulk)

In [ ]:
def create_if_not_exists(file_path):
    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:
            # You can optionally write initial content to the file
            file.write("Wavelength\tQ_e_bulk_0W_18N_bulk\tSSA_bulk_0W_18N_bulk\tg_bulk_0W_18N_bulk\t" + '\n')
            #file.write(f"Q_e_bulk_0W_18N_bulk\t")
            #file.write(f"SSA_bulk_0W_18N_bulk\t")
            #file.write(f"g_bulk_0W_18N_bulk\t\n")
        print(f"File '{file_path}' created.")
    else:
        print(f"File '{file_path}' already exists.")

# Check if dust optical properties txt file already exists
file_path1 = 'RRTM_bulk_dust_optical_properties.txt'
file_path2 = 'RRTM_bulk_dust_optical_properties_{}.txt'.format(wavelength)
create_if_not_exists(file_path1)
create_if_not_exists(file_path2)

In [ ]:
# Assuming Q_e_bulk_0W_18N_bulk, SSA_bulk_0W_18N_bulk, g_bulk_0W_18N_bulk are numpy arrays

# Data to append
data_to_append = np.column_stack((wavelength, Q_e_bulk_0W_18N_bulk, SSA_bulk_0W_18N_bulk, g_bulk_0W_18N_bulk))

# Open the file in append mode and use np.savetxt to append data
with open('bulk_dust_optical_properties_{}.txt'.format(wavelength), 'a') as file:
    np.savetxt(file, data_to_append, fmt='%5.3f', delimiter='\t', header='', comments='')
    
# Open the file in append mode and use np.savetxt to append data
with open('bulk_dust_optical_properties.txt', 'a') as file:
    np.savetxt(file, data_to_append, fmt='%5.3f', delimiter='\t', header='', comments='')